<a href="https://colab.research.google.com/github/samarthk/Learning_pyspark/blob/master/Spark_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Java, Spark, and Findspark
This installs Apache Spark 2.3.1, Java 8, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark


--2020-06-14 12:14:13--  https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230778742 (220M) [application/x-gzip]
Saving to: ‘spark-2.4.1-bin-hadoop2.7.tgz’

spark-2.4.1-bin-had 100%[===================>] 220.09M  23.4MB/s    in 11s     

2020-06-14 12:14:24 (20.3 MB/s) - ‘spark-2.4.1-bin-hadoop2.7.tgz’ saved [230778742/230778742]

sample_data  spark-2.4.1-bin-hadoop2.7	spark-2.4.1-bin-hadoop2.7.tgz


# Set Environment Variables
Set the locations where Spark and Java are installed.

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

# Start a SparkSession
This will start a local Spark session.

In [0]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext

conf = SparkConf().setMaster("local[*]").setAppName('pyspark')
sc = SparkContext(conf=conf)
sqc = SQLContext(sc)

# Use Spark!
That's all there is to it - you're ready to use Spark!

# Programming Ex


Download File

In [0]:
!wget https://data.sfgov.org/api/views/wr8u-xric/rows.csv

In [41]:
!ls -ltr /content/sample_data/

#!head -10000 /content/drive/My\ Drive/Data_Files/Fire_Department_Calls_for_Service_FULL.csv > Fire_Department_Calls_for_Service.csv


total 63512
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root  1706430 Jun 10 16:28 california_housing_train.csv
-rw-r--r-- 1 root root   301141 Jun 10 16:28 california_housing_test.csv
-rw-r--r-- 1 root root 36523880 Jun 10 16:28 mnist_train_small.csv
-rw-r--r-- 1 root root 18289443 Jun 10 16:28 mnist_test.csv
-rw-r--r-- 1 root root     7640 Jun 14 12:17 movies_dataset.csv
-rw-r--r-- 1 root root  3997368 Jun 14 13:03 Fire_Incidents.csv
-rw-r--r-- 1 root root  4191435 Jun 14 13:04 Fire_Department_Calls_for_Service.csv


## San Francisco Fire Incidents (Meetup)

In [44]:
fireServiceCallsDF = pyspark.read.csv('/content/sample_data/Fire_Department_Calls_for_Service.csv', header=True, inferSchema=True)

NameError: ignored